In [2]:
import baostock as bs
import pandas as pd

import datetime
from tqdm import tqdm

In [6]:
# 参数设置
history_days = 15     # 历史股票天数
set_date = ""         # 设置日期

In [8]:
# 计算日期
if not set_date:
    now = datetime.datetime.now()
    if now.hour <= 17 and now.minute <= 30:
        now -= datetime.timedelta(days=1)
else:
    now = datetime.datetime.strptime(set_date, '%Y-%m-%d')
end_date = now.strftime('%Y-%m-%d')
start_date = (now - datetime.timedelta(days=history_days)).strftime('%Y-%m-%d')
print(start_date, end_date)

2021-08-10 2021-08-25


In [9]:
bs.login()

# 查询
stock_rs = bs.query_all_stock(end_date)     # 指定日期的全部股票和指数
stock_df  = stock_rs.get_data()
stock_df  =  stock_df [stock_df ['tradeStatus'] == '1'].reset_index(drop =  True)   # 去除停牌股票

data_df = pd.DataFrame()
structure_9 = []
for row in tqdm(stock_df.itertuples()):
    code = row[1]
    code_name = row[3]
    k_rs = bs.query_history_k_data_plus(code, "date,close", start_date, end_date)
    data_df = k_rs.get_data()
    data_types_dict = {'close': float}
    data_df = data_df.astype(data_types_dict)

    data_df.rolling(9)
    # print(code, data_df.iloc[-2][0] - data_df.iloc[-3][1], data_df.iloc[-2][0] - data_df.iloc[-1][1])
    if len(data_df) >= 3 and data_df.iloc[-2][0] < data_df.iloc[-3][1] and data_df.iloc[-2][0] < data_df.iloc[-1][1]:
        date = data_df.iloc[-1][2]
        close = data_df.iloc[-1][3]
        structure_9.append([code, code_name, date, close])
        
bs.logout()
result = pd.DataFrame(structure_9, columns=['code', 'code_name', 'date', 'close'])
# result = result.reset_index(drop=True)
result.to_csv("结构9买入.csv", index=False)

login success!
logout success!
